# 96. セマンティックセグメンテーション入門

## 学習目標

このノートブックでは、以下を学びます：

1. **セマンティックセグメンテーション**とは何か
2. **画像分類との違い**
3. **密なピクセル予測**の課題
4. **基本的なアプローチ**

## 目次

1. [セマンティックセグメンテーションとは](#section1)
2. [タスクの比較](#section2)
3. [技術的課題](#section3)
4. [基本アプローチ](#section4)
5. [まとめ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import japanize_matplotlib

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

<a id="section1"></a>
## 1. セマンティックセグメンテーションとは

### 定義

**セマンティックセグメンテーション**とは：

> 画像の各ピクセルにクラスラベルを割り当てるタスク

「この画像には猫がいる」ではなく、「このピクセルは猫、このピクセルは背景...」と判断します。

In [ ]:
def visualize_segmentation_concept():
    """セグメンテーションの概念を可視化"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # サンプル画像（簡略化）
    np.random.seed(42)
    img = np.zeros((64, 64, 3))
    
    # 背景（空）
    img[:30, :, :] = [0.5, 0.7, 1.0]  # 青空
    
    # 背景（地面）
    img[30:, :, :] = [0.3, 0.6, 0.3]  # 緑の草
    
    # 物体（猫のシルエット）
    img[20:50, 20:45, :] = [0.6, 0.4, 0.2]  # 茶色の猫
    
    # 入力画像
    axes[0].imshow(img)
    axes[0].set_title('入力画像', fontsize=14)
    axes[0].axis('off')
    
    # セグメンテーションマスク
    mask = np.zeros((64, 64))
    mask[:30, :] = 0  # 空
    mask[30:, :] = 1  # 草
    mask[20:50, 20:45] = 2  # 猫
    
    cmap = plt.cm.colors.ListedColormap(['skyblue', 'lightgreen', 'orange'])
    axes[1].imshow(mask, cmap=cmap)
    axes[1].set_title('セグメンテーションマスク', fontsize=14)
    axes[1].axis('off')
    
    # 凡例
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='skyblue', label='空 (0)'),
        Patch(facecolor='lightgreen', label='草 (1)'),
        Patch(facecolor='orange', label='猫 (2)'),
    ]
    axes[1].legend(handles=legend_elements, loc='upper right')
    
    # オーバーレイ
    axes[2].imshow(img)
    axes[2].imshow(mask, cmap=cmap, alpha=0.5)
    axes[2].set_title('オーバーレイ表示', fontsize=14)
    axes[2].axis('off')
    
    plt.suptitle('セマンティックセグメンテーション：各ピクセルにクラスを割り当て', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_segmentation_concept()

<a id="section2"></a>
## 2. タスクの比較

コンピュータビジョンの主要タスクを比較します。

In [ ]:
def compare_cv_tasks():
    """CVタスクの比較"""
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # サンプル画像
    np.random.seed(42)
    img = np.zeros((64, 64, 3))
    img[:, :, :] = [0.8, 0.9, 1.0]  # 背景
    
    # 2つの物体
    img[10:30, 10:30, :] = [1.0, 0.5, 0.5]  # 赤い四角
    img[35:55, 35:55, :] = [0.5, 0.5, 1.0]  # 青い四角
    
    tasks = [
        ('画像分類', 'この画像には何がある？\n→ 「四角」'),
        ('物体検出', '物体はどこにある？\n→ バウンディングボックス'),
        ('セマンティックセグメンテーション', '各ピクセルは何？\n→ ピクセル単位のラベル'),
        ('インスタンスセグメンテーション', '各物体はどれ？\n→ 物体ごとに区別'),
        ('パノプティックセグメンテーション', '全てを統合\n→ stuff + things'),
    ]
    
    # 1. 画像分類
    axes[0, 0].imshow(img)
    axes[0, 0].set_title('画像分類\n出力: クラスラベル', fontsize=12)
    axes[0, 0].text(32, 70, 'クラス: "四角"', ha='center', fontsize=11)
    axes[0, 0].axis('off')
    
    # 2. 物体検出
    axes[0, 1].imshow(img)
    rect1 = Rectangle((10, 10), 20, 20, linewidth=2, edgecolor='red', facecolor='none')
    rect2 = Rectangle((35, 35), 20, 20, linewidth=2, edgecolor='blue', facecolor='none')
    axes[0, 1].add_patch(rect1)
    axes[0, 1].add_patch(rect2)
    axes[0, 1].set_title('物体検出\n出力: BBox + クラス', fontsize=12)
    axes[0, 1].axis('off')
    
    # 3. セマンティックセグメンテーション
    mask = np.zeros((64, 64))
    mask[10:30, 10:30] = 1
    mask[35:55, 35:55] = 1  # 同じクラス
    
    axes[0, 2].imshow(img)
    axes[0, 2].imshow(mask, alpha=0.5, cmap='Reds')
    axes[0, 2].set_title('セマンティックセグメンテーション\n出力: ピクセルごとのクラス', fontsize=12)
    axes[0, 2].axis('off')
    
    # 4. インスタンスセグメンテーション
    mask_instance = np.zeros((64, 64))
    mask_instance[10:30, 10:30] = 1
    mask_instance[35:55, 35:55] = 2  # 別のインスタンス
    
    axes[1, 0].imshow(img)
    axes[1, 0].imshow(mask_instance, alpha=0.5, cmap='tab10')
    axes[1, 0].set_title('インスタンスセグメンテーション\n出力: 物体ごとに区別', fontsize=12)
    axes[1, 0].axis('off')
    
    # 5. 比較表
    axes[1, 1].axis('off')
    table_data = [
        ['タスク', '出力形式', '粒度'],
        ['分類', 'クラス', '画像単位'],
        ['検出', 'BBox+クラス', '物体単位'],
        ['セマンティック', 'マスク', 'ピクセル単位'],
        ['インスタンス', 'マスク+ID', 'ピクセル単位'],
    ]
    table = axes[1, 1].table(cellText=table_data, loc='center', cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1.5, 2)
    axes[1, 1].set_title('タスク比較', fontsize=12)
    
    # 空のプロット
    axes[1, 2].axis('off')
    axes[1, 2].text(0.5, 0.5, 'セマンティックセグメンテーションは\n\n・ピクセル単位の分類\n・位置情報を完全に保持\n・Dense Prediction', 
                   fontsize=12, ha='center', va='center', transform=axes[1, 2].transAxes,
                   bbox=dict(boxstyle='round', facecolor='lightyellow'))
    
    plt.suptitle('コンピュータビジョンタスクの比較', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

compare_cv_tasks()

<a id="section3"></a>
## 3. 技術的課題

セマンティックセグメンテーションには、画像分類にはない課題があります。

In [ ]:
def explain_challenges():
    """セグメンテーションの課題を説明"""
    print("="*60)
    print("セマンティックセグメンテーションの技術的課題")
    print("="*60)
    
    challenges = [
        ("1. 解像度の維持",
         "画像分類では最終的にGlobal Poolingで空間情報を捨てる",
         "セグメンテーションでは入力と同じ解像度の出力が必要"),
        
        ("2. 局所性と大域性のバランス",
         "正確な境界には局所的な情報が必要",
         "クラス判定には大域的なコンテキストが必要"),
        
        ("3. 計算コスト",
         "高解像度を維持すると計算量が爆発",
         "効率的なアーキテクチャ設計が必要"),
        
        ("4. クラス不均衡",
         "背景ピクセルが圧倒的に多い",
         "適切な損失関数の設計が必要"),
    ]
    
    for name, problem, requirement in challenges:
        print(f"\n{name}")
        print(f"  問題: {problem}")
        print(f"  要件: {requirement}")

explain_challenges()

In [ ]:
def visualize_resolution_problem():
    """解像度維持の問題を可視化"""
    fig, axes = plt.subplots(1, 4, figsize=(18, 4))
    
    # 元画像
    img = np.random.rand(64, 64)
    axes[0].imshow(img, cmap='gray')
    axes[0].set_title('入力\n64×64', fontsize=12)
    axes[0].axis('off')
    
    # CNNによるダウンサンプリング
    from scipy.ndimage import zoom
    
    img_down1 = zoom(img, 0.5)  # 32x32
    axes[1].imshow(img_down1, cmap='gray')
    axes[1].set_title('Conv+Pool ×2\n32×32', fontsize=12)
    axes[1].axis('off')
    
    img_down2 = zoom(img, 0.25)  # 16x16
    axes[2].imshow(img_down2, cmap='gray')
    axes[2].set_title('Conv+Pool ×4\n16×16', fontsize=12)
    axes[2].axis('off')
    
    # 問題：これを64x64に戻す必要がある
    img_up = zoom(img_down2, 4)  # 64x64に戻す
    axes[3].imshow(img_up, cmap='gray')
    axes[3].set_title('アップサンプル\n64×64（情報損失）', fontsize=12)
    axes[3].axis('off')
    
    plt.suptitle('解像度の問題：ダウンサンプリングで詳細が失われる', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_resolution_problem()

<a id="section4"></a>
## 4. 基本アプローチ

セグメンテーションの基本的なアプローチを紹介します。

In [ ]:
def visualize_fcn_concept():
    """FCN（Fully Convolutional Network）の概念"""
    fig, ax = plt.subplots(figsize=(16, 8))
    
    # エンコーダ
    encoder_layers = [(0.05, 0.5, 0.08, 0.4, '入力\n64×64'),
                      (0.18, 0.55, 0.06, 0.3, '32×32'),
                      (0.28, 0.6, 0.05, 0.2, '16×16'),
                      (0.38, 0.65, 0.04, 0.1, '8×8')]
    
    for x, y, w, h, label in encoder_layers:
        rect = Rectangle((x, y - h/2), w, h, facecolor='lightblue', edgecolor='blue')
        ax.add_patch(rect)
        ax.text(x + w/2, y - h/2 - 0.05, label, ha='center', fontsize=9)
    
    ax.text(0.22, 0.85, 'エンコーダ\n（特徴抽出）', ha='center', fontsize=12, fontweight='bold')
    
    # デコーダ
    decoder_layers = [(0.55, 0.65, 0.04, 0.1, '8×8'),
                      (0.65, 0.6, 0.05, 0.2, '16×16'),
                      (0.78, 0.55, 0.06, 0.3, '32×32'),
                      (0.9, 0.5, 0.08, 0.4, '出力\n64×64')]
    
    for x, y, w, h, label in decoder_layers:
        rect = Rectangle((x, y - h/2), w, h, facecolor='lightgreen', edgecolor='green')
        ax.add_patch(rect)
        ax.text(x + w/2, y - h/2 - 0.05, label, ha='center', fontsize=9)
    
    ax.text(0.75, 0.85, 'デコーダ\n（アップサンプル）', ha='center', fontsize=12, fontweight='bold')
    
    # 矢印
    for i in range(3):
        ax.annotate('', xy=(encoder_layers[i+1][0], encoder_layers[i+1][1]),
                   xytext=(encoder_layers[i][0] + encoder_layers[i][2], encoder_layers[i][1]),
                   arrowprops=dict(arrowstyle='->', color='blue'))
    
    ax.annotate('', xy=(decoder_layers[0][0], decoder_layers[0][1]),
               xytext=(encoder_layers[-1][0] + encoder_layers[-1][2], encoder_layers[-1][1]),
               arrowprops=dict(arrowstyle='->', color='gray', lw=2))
    
    for i in range(3):
        ax.annotate('', xy=(decoder_layers[i+1][0], decoder_layers[i+1][1]),
                   xytext=(decoder_layers[i][0] + decoder_layers[i][2], decoder_layers[i][1]),
                   arrowprops=dict(arrowstyle='->', color='green'))
    
    # 説明
    ax.text(0.5, 0.15, 'FCN: 全結合層を畳み込みに置き換え\n→ 任意の入力サイズに対応可能', 
           ha='center', fontsize=12, transform=ax.transAxes,
           bbox=dict(boxstyle='round', facecolor='lightyellow'))
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('Encoder-Decoder アーキテクチャ', fontsize=16, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_fcn_concept()

<a id="summary"></a>
## 5. まとめ

### 学んだこと

1. **セマンティックセグメンテーション**: 各ピクセルにクラスを割り当てる
2. **課題**: 解像度維持、局所性と大域性のバランス
3. **基本アプローチ**: Encoder-Decoder構造

### 次のノートブック

次のノートブックでは、**U-Net**アーキテクチャについて詳しく学びます。